## list value update for numerical sequence without changing index


In [3]:
import numpy as np

In [2]:
labels = [1, 2, 1, 4, 0, 6]

In [3]:
unique_seq, unique_index = np.unique(labels, return_index=True)
for seq, index in zip(unique_seq, unique_index):
    print('{}--{}'.format(index, seq))

4--0
0--1
1--2
3--4
5--6


In [4]:
sort = sorted(unique_index)
new = [labels[i] for i in sort]
pairs = {v : k for k,v in enumerate(new)}
pairs

{1: 0, 2: 1, 4: 2, 0: 3, 6: 4}

In [5]:
new_label = []
for v in labels:
    new_label.append(pairs[v])
print(new_label)

[0, 1, 0, 2, 3, 4]


In [6]:
def sort_speakers(labels):
    _, index = np.unique(labels, return_index=True)
    pairs = {labels[v]:k for k,v in enumerate(sorted(index))}
    sorted_labels = [pairs[label] for label in labels]
    return sorted_labels

In [7]:
new_labels = sort_speakers(labels)
print(f"Old labels\n{labels}\n")
print(f"New labels\n{new_labels}")

Old labels
[1, 2, 1, 4, 0, 6]

New labels
[0, 1, 0, 2, 3, 4]


## trimming audio with duration(bytes data)

In [4]:
from tools import wavSplit
import os
audio_path = r'/home/zmh/Desktop/HDD/Workspace/my_github/Speech-Diarization/test-data'
wavfile = "Mark Zuckerberg's 2004 Interview See How Far He And Facebook Have Come - YouTube.wav"


In [5]:
audio, sample_rate, audio_length = wavSplit.read_wave(os.path.join(audio_path, wavfile))
print(len(audio),',' ,audio_length, 'minutes')

7176638 , 224.2699375 minutes


In [6]:
frames = list(wavSplit.frame_generator(30, audio, sample_rate))

In [7]:
len(frames)

7475

In [8]:
frame = frames[0]

In [9]:
frame.duration

0.03

In [10]:
frames[2].bytes.__len__()

960

In [11]:
def write_wave(audio, wav_name, sample_rate):
    """Writes a .wav file.
    Takes path, PCM audio data, and sample rate.
    """
    with contextlib.closing(wave.open(wav_name, 'wb')) as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(sample_rate)
        wf.writeframes(audio)


In [32]:
def segment_bytes(audio, sample_rate, time_limit=60): 
    frame_byte_count = int(sample_rate * time_limit * 2)
    offset = 0 
    while offset + frame_byte_count -1 < len(audio):
        yield audio[offset:offset + frame_byte_count] 
        offset += frame_byte_count
    yield audio[offset:]

In [33]:
gen_bytes = list(segment_bytes(audio, sample_rate=16000, time_limit=60))
len(gen_bytes)

4

In [20]:
import contextlib
import wave
for i, byte in enumerate(gen_bytes):
    write_wave(byte, 'test-{}.wav'.format(i), 16000)

In [26]:
#load variable with pickle
import pickle
with open('facebook_interview_segments.pkl', 'rb') as f:
    data = pickle.load(f)

data[6].bytes.__len__()/32000

63.45

In [27]:
# mark zuckerberg audio segment
test = data[6]

In [31]:
gen_bytes = list(segment_bytes(test.bytes, sample_rate=16000, time_limit=60))
for i, byte in enumerate(gen_bytes):
    write_wave(byte, 'seg-{}.wav'.format(i), 16000)

4